<a href="https://colab.research.google.com/github/marcellosusanto11/predict_ncaa_tournament/blob/master/Scraper/scrape_vegas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# Set working directory
%cd drive/My\ Drive/Colab\ Notebooks/Tugas\ Akhir/

/content/drive/My Drive/Colab Notebooks/Tugas Akhir


In [3]:
# Install package
!pip install tqdm

In [0]:
# Setting general package
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

# Setting specific package
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm

from matplotlib.pylab import rcParams
# For every plotting cell use this
# fig, axes = plt.subplots()
rcParams['figure.figsize'] = [10,5]
plt.style.use('fivethirtyeight') 
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

## Scraping Odds Data
Source : covers.com

covers.com provide vegas point spread data starting from 2010-2011 NCAA Regular Season until 2020. Below the list of variable that we can get from scraping :
 - Date
 - HomeTeam/AwayTeam
 - HomeScore/AwayScore
 - HomeConf/AwayConf
 - Competiton Type
 - Odd

In [0]:
# Function to make the correct Date
def convert_date(num) :
  if num < 10 :
    conv = '0' + str(num)
  else :
    conv = str(num)

  return conv

In [6]:
# -- Scraping program for vegas point spread data

# Define list to make the dataframe
dict_df = {'Date':[], 'HomeTeam':[], 'AwayTeam':[], 'HomeConf':[], 'AwayConf':[], 'HomeScore':[], 'AwayScore':[], 'Competition':[], 'Odd':[]}
list_col = list(dict_df.keys())

# Define base url 
base_url = 'https://www.covers.com/Sports/NCAAB/Matchups?selectedDate='

# Iteration to make Date string
# 8 November 2010 is a Date where the 2010-2011 season start
for date in tqdm(pd.date_range(pd.to_datetime('2010-11-08'), pd.datetime.today()).date) :

  # Complete the base url
  date = str(date)
  url = base_url + date

  # Get the HTML from the page
  try :
    f = requests.get(url)
  except :
    continue
  soup = BeautifulSoup(f.text)

  # Iteration for scraping
  for match in soup.find_all('div', {"class": "cmg_matchup_game_box cmg_game_data"}) :
    if match['data-game-odd'] != '' :
      try :
        v_date = pd.to_datetime(date)
        v_hometeam = match['data-home-team-fullname-search']
        v_awayteam = match['data-away-team-fullname-search']
        v_homescore = match['data-home-score']
        v_awayscore = match['data-away-score']
        v_homeconf = match['data-home-conference']
        v_awayconf = match['data-away-conference']
        v_comp = match['data-competition-type']
        v_odd = match['data-game-odd']
      except :
        continue

      # Compile all the variable we get into list
      list_df = [v_date, v_hometeam, v_awayteam, v_homescore, v_awayscore, v_homeconf, v_awayconf, v_comp, v_odd]

      # Put it into dictionary
      for i in range(len(list_df)) :
        dict_df[list_col[i]].append(list_df[i])

# Make the dataframe
df_vegas = pd.DataFrame(dict_df)

100%|██████████| 3385/3385 [1:57:50<00:00,  1.63s/it]


In [7]:
df_vegas.head()

,Date,HomeTeam,AwayTeam,HomeConf,AwayConf,HomeScore,AwayScore,Competition,Odd
0,2011-04-05,St. John's,William & Mary,74,59,Big East,Colonial Athletic Association,Regular Season,-7.5
1,2011-04-05,Mississippi State,Eastern Kentucky,76,66,Southeastern,Ohio Valley,Regular Season,-16.5
2,2011-04-05,Arizona,Valparaiso,73,64,Pacific-12,Missouri Valley,Regular Season,-14.5
3,2011-04-06,St. John's,William & Mary,74,59,Big East,Colonial Athletic Association,Regular Season,-7.5
4,2011-04-06,Mississippi State,Eastern Kentucky,76,66,Southeastern,Ohio Valley,Regular Season,-16.5


In [0]:
# Simpan dataframenya
df_vegas.to_csv('Data LasVegas/vegas.csv', index=False)